# Sentiment Analysis of Yelp reviews: Word Embeddings and LSTM

In this notebook, we implement a sentiment analysis to associate stars (from 1 to 5) to the reviews of the [YelpReviewFull dataset](https://pytorch.org/text/stable/datasets.html#yelpreviewfull). 
This dataset consists of reviews from Yelp, and is extracted from the Yelp Dataset Challenge 2015 data. 
A data point of this dataset comprises a review's text and the corresponding label (1 to 5 stars).

For this, we will first use ..WORD EMBEDDINGS... TO...    [ResNet](https://arxiv.org/abs/1512.03385)

Then, we will use XXX... to...    [ResNet](https://arxiv.org/abs/1512.03385)

We achieve a XX% ...



In [13]:
%%bash
pip install torch==1.11.0
pip install folium==0.2.1
pip install torchdata
pip install datasets

  Using cached datasets-2.2.2-py3-none-any.whl (346 kB)
  Using cached huggingface_hub-0.7.0-py3-none-any.whl (86 kB)
  Using cached pyarrow-8.0.0-cp38-cp38-macosx_10_13_x86_64.whl (22.4 MB)
  Using cached fsspec-2022.5.0-py3-none-any.whl (140 kB)
  Using cached aiohttp-3.8.1-cp38-cp38-macosx_10_9_x86_64.whl (574 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached multiprocess-0.70.13-py38-none-any.whl (131 kB)
  Using cached xxhash-3.0.0-cp38-cp38-macosx_10_9_x86_64.whl (34 kB)
  Using cached filelock-3.7.1-py3-none-any.whl (10 kB)
  Using cached PyYAML-6.0-cp38-cp38-macosx_10_9_x86_64.whl (192 kB)
  Using cached yarl-1.7.2-cp38-cp38-macosx_10_9_x86_64.whl (121 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 kB)
  Using cached frozenlist-1.3.0-cp38-cp38-macosx_10_9_x86_64.whl (36 kB)
  Using cached multidict-6.0.2-cp38-cp38-macosx_10_9_x8

## Data pre-processing: Tokenization

To begin, we download the data with the [Hugging Face](https://huggingface.co/) library datasets. 

(The direct loading from torchtext.datsets is impossible for now, because of a bug that will be corrected in the next version of PyTorch.)

In [14]:
import torch
import torch.nn as nn

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [15]:
import torchtext.transforms as T
from torch.hub import load_state_dict_from_url

padding_idx = 1
bos_idx = 0
eos_idx = 2
max_seq_len = 256
xlmr_vocab_path = r"https://download.pytorch.org/models/text/xlmr.vocab.pt"
xlmr_spm_model_path = r"https://download.pytorch.org/models/text/xlmr.sentencepiece.bpe.model"

text_transform = T.Sequential(
    T.SentencePieceTokenizer(xlmr_spm_model_path),
    T.VocabTransform(load_state_dict_from_url(xlmr_vocab_path)),
    T.Truncate(max_seq_len - 2),
    T.AddToken(token=bos_idx, begin=True),
    T.AddToken(token=eos_idx, begin=False),
)

In [16]:
from datasets import load_dataset
train_data = load_dataset("yelp_review_full", split="train")
test_data = load_dataset("yelp_review_full", split="test")

train_iter = iter(train_data)
test_iter = iter(test_data)

In [17]:

from torch.utils.data import DataLoader
batch_size = 16



train_datapipe = train_iter
dev_datapipe = test_iter

# Transform the raw dataset using non-batched API (i.e apply transformation line by line)
train_datapipe = train_datapipe.map(lambda x: (text_transform(x[0]), x[1]))
train_datapipe = train_datapipe.batch(batch_size)
train_datapipe = train_datapipe.rows2columnar(["token_ids", "target"])
train_dataloader = DataLoader(train_datapipe, batch_size=None)

dev_datapipe = dev_datapipe.map(lambda x: (text_transform(x[0]), x[1]))
dev_datapipe = dev_datapipe.batch(batch_size)
dev_datapipe = dev_datapipe.rows2columnar(["token_ids", "target"])
dev_dataloader = DataLoader(dev_datapipe, batch_size=None)

AttributeError: 'generator' object has no attribute 'map'

In [7]:
num_classes = 2
input_dim = 768

from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER
classifier_head = RobertaClassificationHead(num_classes=num_classes, input_dim=input_dim)
model = XLMR_BASE_ENCODER.get_model(head=classifier_head)
model.to(DEVICE)

RobertaModel(
  (encoder): RobertaEncoder(
    (transformer): TransformerEncoder(
      (token_embedding): Embedding(250002, 768, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (dropout): Dropout(p=0.1, inplace=False)
          (attention): MultiheadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (input_projection): Linear(in_features=768, out_features=2304, bias=True)
            (output_projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (residual_mlp): ResidualMLP(
            (mlp): Sequential(
              (0): Linear(in_features=768, out_features=3072, bias=True)
              (1): GELU()
              (2): Dropout(p=0.1, inplace=False)
              (3): Linear(in_features=3072, out_features=768, bias=True)
              (4): Dropout(p=0.1, inplace=False)
            )
          )
          (attention_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)


In [8]:
num_classes = 2
input_dim = 768

from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER

classifier_head = RobertaClassificationHead(num_classes=num_classes, input_dim=input_dim)
model = XLMR_BASE_ENCODER.get_model(head=classifier_head)
model.to(DEVICE)

RobertaModel(
  (encoder): RobertaEncoder(
    (transformer): TransformerEncoder(
      (token_embedding): Embedding(250002, 768, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (dropout): Dropout(p=0.1, inplace=False)
          (attention): MultiheadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (input_projection): Linear(in_features=768, out_features=2304, bias=True)
            (output_projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (residual_mlp): ResidualMLP(
            (mlp): Sequential(
              (0): Linear(in_features=768, out_features=3072, bias=True)
              (1): GELU()
              (2): Dropout(p=0.1, inplace=False)
              (3): Linear(in_features=3072, out_features=768, bias=True)
              (4): Dropout(p=0.1, inplace=False)
            )
          )
          (attention_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)


In [9]:
import torchtext.functional as F
from torch.optim import AdamW

learning_rate = 1e-5
optim = AdamW(model.parameters(), lr=learning_rate)
criteria = nn.CrossEntropyLoss()


def train_step(input, target):
    output = model(input)
    loss = criteria(output, target)
    optim.zero_grad()
    loss.backward()
    optim.step()


def eval_step(input, target):
    output = model(input)
    loss = criteria(output, target).item()
    return float(loss), (output.argmax(1) == target).type(torch.float).sum().item()


def evaluate():
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    counter = 0
    with torch.no_grad():
        for batch in dev_dataloader:
            input = F.to_tensor(batch["token_ids"], padding_value=padding_idx).to(DEVICE)
            target = torch.tensor(batch["target"]).to(DEVICE)
            loss, predictions = eval_step(input, target)
            total_loss += loss
            correct_predictions += predictions
            total_predictions += len(target)
            counter += 1

    return total_loss / counter, correct_predictions / total_predictions

In [10]:
num_epochs = 1

for e in range(num_epochs):
    for batch in train_dataloader:
        input = F.to_tensor(batch["token_ids"], padding_value=padding_idx).to(DEVICE)
        target = torch.tensor(batch["target"]).to(DEVICE)
        train_step(input, target)

    loss, accuracy = evaluate()
    print("Epoch = [{}], loss = [{}], accuracy = [{}]".format(e, loss, accuracy))

RuntimeError: Internal error: headers don't contain content-disposition.